In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd
import numpy as np
import cx_Oracle

#-*- coding:utf-8 -*-

client_id="MV6NBr8jD4HVKrnXIT6Z"
client_secret="KB0BVduyM5"

In [2]:
#[CODE 1]
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [3]:
#[CODE 2]
def getNaverSearchResult(sNode, search_text, page_start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text), page_start, display)
    #parameters = "?query=%s" % urllib.parse.quote(search_text)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [4]:
#[CODE 3]
def getJsonData(jsonSearch,search_text):
    jsonResult=[]
    for post in jsonSearch['items']:
        title = post['title']
        description = post['description']
        org_link = post['originallink']
        link = post['link']

    #Tue, 14 Feb 2017 18:46:00 +0900
        pDate = datetime.datetime.strptime(post['pubDate'],'%a, %d %b %Y %H:%M:%S +0900')
        pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
        jsonResult.append({'title':title, 'description': description,
                    'org_link':org_link, 'link': org_link, 
                    'pDate':pDate})
    
    with open('%s_naver.json' % (search_text), 'w', encoding='utf8') as outfile:
        retJson = json.dumps(jsonResult,
                        indent=4, sort_keys=True,
                        ensure_ascii=False)
        outfile.write(retJson)
        
    print ('%s_naver.json SAVED' % (search_text))
        
    return jsonResult

In [8]:
#[CODE4]
def getDataFrame(jsonSearch,search_text):
    
    titles=[];des=[];org_links=[];links=[];pDate=[]
    for row in jsonSearch['items']:
        titles.append(row['title'].strip().replace(u'\xa0',''))
        des.append(row['description'].strip().replace(u'\xa0',''))
        org_links.append(row['originallink'])
        links.append(row['link'])

        mDate = datetime.datetime.strptime(row['pubDate'],'%a, %d %b %Y %H:%M:%S +0900')
        mDate = mDate.strftime('%Y-%m-%d %H:%M:%S')
        pDate.append(mDate)
    
    df=pd.DataFrame({'title':titles,'description':des,'org_link':org_links,'link':links,'date':pDate})
    df.to_csv('%s_naver.csv'%(search_text),index=False)
    print ('%s_naver.csv SAVED' % (search_text))
    return df

def oracle_save(df):
    conn=cx_Oracle.connect('pgm/1234@localhost:1521/orcl')
    cursor=conn.cursor()
    sql="insert into search_data values(:1,:2,:3,:4,:5)"
    rows=[tuple(x) for x in df.values]
    for row in rows:
        cursor.execute(sql,row)
    conn.commit()
    cursor.close()
    conn.close()

def main():
    jsonResult = []
    # 'news', 'blog', 'cafearticle'
    sNode = 'news'
    search_text = '지소미아'
    display_count = 100
    
    
    jsonSearch = getNaverSearchResult(sNode, search_text, 1, display_count)
    jsonResult=getJsonData(jsonSearch, search_text)
    
    #while ((jsonSearch != None) and (jsonSearch['display'] != 0)):
    df=getDataFrame(jsonSearch, search_text)
    
    oracle_save(df)
    
    
    
    
if __name__ == '__main__':
    main()

[2019-11-21 12:32:50.973415] Url Request Success
지소미아_naver.json SAVED
지소미아_naver.csv SAVED
